In [1]:
import numpy as np 
import os
import glob
import shutil
import skimage.io as io
import skimage.transform as trans
import matplotlib.pyplot as plt
import cv2
from keras.callbacks import ModelCheckpoint


2022-02-25 15:09:36.465995: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /media/sda2/0installed/anaconda3/envs/test/lib/python3.9/site-packages/cv2/../../lib64:
2022-02-25 15:09:36.466018: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
diretorio = "/media/sda2/home/j/Downloads/tests/"
saida = "/media/sda2/home/j/Downloads/tests/recortes_tmp/"

dataset_name = "Base_A/"
dataset_img = "corte_2360"

do_cut = False
#do_cut = True

if(do_cut):
    img = io.imread(diretorio + dataset_img + ".tif")
    mask = io.imread(diretorio + dataset_img + "_linhas.tif")
    [lin, col, ch] = img.shape
    print(lin, col, ch)
    square = 256
    offset = 256
    img_number = 0
    for i in range(0, lin, offset):
        for j in range(0, col, offset):           
            if(i+square > lin or j+square > col):
                continue
            alpha = img[i:i+square, j:j+square, 3]
            n_white_pix = np.sum(alpha == 255)
            print(n_white_pix)
            
            if(n_white_pix / (square*square) > 0.8):
                image_without_alpha = img[i:i+square, j:j+square, :3]
                cut_mask = mask[i:i+square, j:j+square]
                io.imsave(saida + dataset_name + "/" + str(img_number) + ".png",
                          image_without_alpha, check_contrast=False)
                io.imsave(saida + dataset + "_mask/" + str(img_number) + ".png",
                          cut_mask, check_contrast=False)
                img_number = img_number + 1


In [3]:
diretorio = "/media/sda2/home/j/Downloads/tests/recortes_tmp/"
saida = "/media/sda2/home/j/Downloads/tests/recortes_tmp/folds/"

dataset_name = "Base_A"

!mkdir -p $saida

nr_files = len(glob.glob(diretorio + dataset_name + "/*.png"))
#print(nr_files)

folds = 10 #10
offset = 0
nr_test = int(nr_files / folds)
nr_train = nr_files - nr_test

#do_folds = False
do_folds = True

if(do_folds):
    for atual_fold in range(0, 2): #folds = 10
        print("atual_fold:", atual_fold, saida + str(atual_fold + 1))

        try:            
            os.mkdir(saida + str(atual_fold + 1))
            os.mkdir(saida + str(atual_fold + 1) + "/train")
            os.mkdir(saida + str(atual_fold + 1) + "/test")
            os.mkdir(saida + str(atual_fold + 1) + "/train/images")
            os.mkdir(saida + str(atual_fold + 1) + "/train/labels")
            os.mkdir(saida + str(atual_fold + 1) + "/test/images")
            os.mkdir(saida + str(atual_fold + 1) + "/test/labels")
        except:
            None

        aux_test = 0
        aux_train = 0
        aux_contador = 0
        for name in sorted(glob.glob(diretorio + dataset_name + "/*.png")):
            # nesse caso to montando test
            if aux_contador >= offset:
                if aux_contador <= offset + nr_test:
                    shutil.copy(name, saida+str(atual_fold + 1)+ "/test/images/" + os.path.basename(name))
                else:
                    shutil.copy(name, saida+str(atual_fold + 1)+ "/train/images/" + os.path.basename(name))
            else:
                shutil.copy(name, saida+str(atual_fold + 1)+ "/train/images/" + os.path.basename(name))

            aux_contador = aux_contador + 1

        aux_test = 0
        aux_train = 0
        aux_contador = 0
        for name in sorted(glob.glob(diretorio + dataset_name + "_mask/*.png")):
            # nesse caso to montando test
            if aux_contador >= offset:
                if aux_contador <= offset + nr_test:
                    shutil.copy(name, saida+str(atual_fold + 1) + "/test/labels/" + os.path.basename(name))
                else:
                    shutil.copy(name, saida+str(atual_fold + 1) +"/train/labels/" + os.path.basename(name))
            else:
                shutil.copy(name, saida+str(atual_fold + 1) + "/train/labels/" + os.path.basename(name))

            aux_contador = aux_contador + 1
        offset = offset + (nr_test + 1)


atual_fold: 0 /media/sda2/home/j/Downloads/tests/recortes_tmp/folds/1
atual_fold: 1 /media/sda2/home/j/Downloads/tests/recortes_tmp/folds/2
